In [1]:
import pandas as pd
import numpy

In [2]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [3]:
file = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=100000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [4]:
file = file.dropna()

In [5]:
file.shape

(99981, 11)

In [7]:
def get_copurchase_df(df):
    count_size = df.index.unique()
    size_df = df.groupby('交易id').size()
    size_df = size_df[size_df < 2]
    return df.drop(list(size_df.index), axis=0)

In [8]:
data = get_copurchase_df(file)

In [9]:
data.head()

,資料日期與時間,餐別帶,縣市別,店舖代號,主商圈,品號-品名稱,群號-群名稱,單品名稱,銷售數量,銷售單價,交易金額
交易id,,,,,,,,,,,
01637920171201184556000112534201,2017-12-01 18:45:56,晚餐時間帶,台中市,11148,住宅型,72-家庭雜貨,722-家雜用品,（新）銷售用購物袋１８號袋,1,1.0,1.0
01614720171201163042000114405402,2017-12-01 16:30:42,下午茶時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,飲冰室茶集綠奶茶,1,25.0,25.0
01384020171201130558000119295301,2017-12-01 13:05:58,午餐時間帶,彰化縣,13840,文教型,12-調理麵,122-熱食麵２配,沙茶牛肉炒麵,1,69.0,69.0
01475120171201153506000111697202,2017-12-01 15:35:06,下午茶時間帶,新竹市,4151,工業型,29-冷藏飲料,291-冷藏茶飲料,冷泡茶台茶１２號紅茶,1,25.0,21.0
01564720171201180924000113751602,2017-12-01 18:09:24,晚餐時間帶,台中市,3860,住宅型,34-健康飲料,343-水,多喝水２Ｌ,2,35.0,49.0


In [10]:
def aggregate(df):
    dic = {}
    for index, row in df.iterrows():
        if index not in dic:
            dic[index] = []
        dic[index].append({
            'name': row['單品名稱'],
            'amount': row['銷售單價']
        })
    dic = { index : value for index, value in dic.items() if len(value) > 1}
    return dic

In [11]:
purchase_list = aggregate(data)

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['pn']

In [3]:
purchase_list = list(db['transactions'].find({'items.1': { '$exists': True } }, projection=['items']))

In [45]:
from pprint import pprint
for purchase in purchase_list:
    for item in purchase['items']:
        if item['單品名稱'] == '雅虎拍賣寄件':
            pprint(purchase)
            break

{'_id': ObjectId('5ccfa08b578a4bcd0c00e132'),
 'items': [{'amount': 0.0,
            '品號-品名稱': '00-新型態代收',
            '單品名稱': '雅虎拍賣寄件',
            '群號-群名稱': '26-店到店',
            '銷售單價': 0.0},
           {'amount': 120.0,
            '品號-品名稱': '89-資訊商品',
            '單品名稱': '店到店雅虎拍賣手續費',
            '群號-群名稱': '897-資訊服務手續費',
            '銷售單價': 60.0}]}
{'_id': ObjectId('5ccfa08b578a4bcd0c00e784'),
 'items': [{'amount': 0.0,
            '品號-品名稱': '00-新型態代收',
            '單品名稱': '雅虎拍賣寄件',
            '群號-群名稱': '26-店到店',
            '銷售單價': 0.0},
           {'amount': 0.0,
            '品號-品名稱': '00-新型態代收',
            '單品名稱': '露天寄件Ｗ',
            '群號-群名稱': '26-店到店',
            '銷售單價': 0.0},
           {'amount': 60.0,
            '品號-品名稱': '89-資訊商品',
            '單品名稱': '店到店雅虎拍賣手續費',
            '群號-群名稱': '897-資訊服務手續費',
            '銷售單價': 60.0}]}
{'_id': ObjectId('5ccfa08b578a4bcd0c00f912'),
 'items': [{'amount': 60.0,
            '品號-品名稱': '89-資訊商品',
            '單品名稱': '店到店雅虎拍賣手續費',


In [5]:
len(purchase_list)

23288

# Network Analysis

In [64]:
from itertools import filterfalse, combinations

def find_edges_in_list(itemsets):
    result = []
    return combinations(itemsets, 2)

def convert(purchase_list, support):
    result = {}
    nodes = set()
    for transaction in purchase_list:
        itemsets = transaction['items']
        if len(itemsets) > 1:
            edge_list = list(find_edges_in_list(itemsets))
            length = len(edge_list)
            for edge_dict_tuple in edge_list:
                edge = tuple([dic['單品名稱'] for dic in edge_dict_tuple])
                weight = sum([dic['amount'] for dic in edge_dict_tuple]) / length
                if edge in result:
                    result[edge]['count'] += 1
                    result[edge]['weight'] += weight
                else:
                    result[edge] = {}
                    result[edge]['count'] = 1
                    result[edge]['weight'] = weight
    for key in list(result.keys()):
        if result[key]['count'] < support:
            del result[key]
    for items in result.keys():
        for item in items:
            if item not in nodes:
                nodes.add(item)
    return (nodes, result)

In [65]:
nodes, edges_dict = convert(purchase_list, 8)

In [66]:
print('Node number: {}\nEdge number: {}'.format(len(nodes), len(edges_dict)))

Node number: 194
Edge number: 453


In [67]:
import igraph

In [68]:
g = igraph.Graph()

In [69]:
for node in nodes:
    g.add_vertex(node)

In [70]:
len(g.vs)

194

In [71]:
for edge, attrs in edges_dict.items():
    weight = attrs['weight'] if attrs['weight'] > 0 else 1
    g.add_edge(edge[0], edge[1], weight=weight)

In [72]:
g.simplify(combine_edges={ "weight": "sum" })
g.to_undirected(mode='mutual', combine_edges={ "weight": "sum" })

In [73]:
communities = g.community_fastgreedy('weight')

In [74]:
cluster = communities.as_clustering()

In [82]:
dics = []
for subgraph in cluster.subgraphs():
    nums = len(subgraph.vs)
    weight_sum = sum([edge['weight'] for edge in subgraph.es]) * (nums) / nums / (nums + 1)
    comm_name = [ node['name'] for node in subgraph.vs]
    dic = {
        'weight': weight_sum,
        'comms': ' '.join(comm_name)
    }
    dics.append(dic)
a = sorted(dics, key=lambda x : x['weight'], reverse=True)
for dic in a:
    print(dic['comms'])
    print("===========")

戰禍邪神第１２章 戰禍邪神第１１章
ＦＰ店到店 店到店ＦＰ手續費
紅標料理米酒 空瓶回收（銷售用）
促銷券０６ 頑皮滷蛋－原味 Ｃｒｅａｍ－Ｏ黑巧克力三明治餅 伊藤園蘋果紅茶
冰拿鐵大杯 熱拿鐵大杯
辣味ＹＵＲＯＣＫ魚薯條無 原味ＹＵＲＯＣＫ魚薯條無
雅虎拍賣寄件 店到店雅虎拍賣手續費
大口奶油蕈菇起司雞排飯糰 熱美式中杯 大口法香烤雞飯糰 熱拿鐵中杯 綠豆沙牛乳 特濃抹茶拿鐵
蕃薯（３０元） 義美奶茶 黑胡椒熱狗 蕃薯（２５元） 鮮奶茶 全家熱狗麵包 頂級鮮奶優格－莓果穀物脆片 商店街取件 特濃黑可可 特濃咖啡拿鐵 統一陽光無糖高纖豆漿 經典原味熱狗 爆濃起司熱狗 熱拿鐵小杯 蕃薯（１５元） 香蕉單入 茶葉蛋（銷售用） 養樂多 簡單點無加糖優酪乳 肉鬆起酥麵包 蕃薯（２０元） 蝦皮取件Ｃ 簡單點原味優酪乳 義美古早傳統無糖豆奶
海鮮魚卵棒 蟹肉糰子 筊白筍 關東煮本舖拉麵 旗魚黑輪 鮮香菇 特級花枝丸 白玉蘿蔔 手工高麗菜捲 杏鮑菇 千層玉子燒 黃金厚切魚板 日式黑輪 野菜多多魚餅 黃金魚豆腐 讚岐烏龍麵 究極味付蛋 王子麵 關東煮本舖冬粉
光泉米漿 光泉無加糖鮮豆漿 ＬＣＡ活菌原味發酵乳 肉鬆飯糰 林鳳營全脂鮮乳 鮪魚飯糰 雪花蛋糕
台鐵取票 台鐵手續費
合庫代１５ 代收手續費１５ 中信外１５ 玉山淘寶款 雅虎拍賣繳費
高鐵手續費 高鐵取票
促銷券１０ ＭＭ花生巧克力
聯合報 自由時報 蘋果日報
寶物交易代 代收手續費２５
優格軟糖（Ｐｅａｃｈ） 優格軟糖（Ｏｒａｎｇｅ） 奶香綠茶３３０ＭＬ 代收折價卷 超涼口香糖（袋） 潤喉糖－蜂蜜檸檬 ＡＷ極酷嗆涼紫冰野莓口香糖
紫米紅豆湯圓 （新）銷售用購物袋１８號袋 蔥爆牛肉燴飯 金牌台啤罐裝（６入） 純喫茶紅茶 七星１０毫克硬盒香煙 日式蒜香燒豚飯 冷泡茶冰釀烏龍 台鹽海洋鹼性離子水 金蘋果調味乳蘋果風味 麥香紅茶ＴＰ３００ 可口可樂ＰＥＴ 泡沫綠茶ＴＰ３００ 統一大布丁（雞蛋口味） 麥香錫蘭奶茶 鹼性離子水 金牌台灣啤酒５００ＭＬ 飲冰室茶集紅奶茶 伯朗咖啡 飲冰室茶集綠奶茶 天然水 奶油雞排歐姆蛋燴飯 冷泡茶冷萃綠茶無糖 純喫茶綠茶６５０ｍｌ 衛生冰 （新）４５號銷售用購物袋 麥香阿薩姆奶茶 全家克林姆麵包 金牌台灣啤酒 泡沫紅茶ＴＰ３００ 麥香綠茶ＴＰ３００ 雲絲頓紅１０毫克香煙 茶裏王日式綠茶 純喫茶無糖綠茶６

In [25]:
cluster.membership

[0,
 1,
 1,
 2,
 3,
 4,
 5,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 11,
 1,
 12,
 13,
 14,
 15,
 15,
 4,
 15,
 4,
 16,
 5,
 15,
 8,
 17,
 12,
 18,
 1,
 1,
 6,
 19,
 20,
 18,
 4,
 4,
 6,
 18,
 18,
 21,
 19,
 4,
 1,
 15,
 18,
 4,
 22,
 18,
 6,
 6,
 11,
 1,
 4,
 1,
 23,
 24,
 18,
 25,
 2,
 15,
 3,
 26,
 4,
 0,
 27,
 1,
 15,
 1,
 8,
 4,
 1,
 15,
 6,
 18,
 18,
 10,
 18,
 7,
 8,
 18,
 1,
 28,
 18,
 29,
 15,
 1,
 30,
 31,
 1,
 28,
 18,
 23,
 4,
 32,
 16,
 15,
 4,
 18,
 1,
 29,
 15,
 4,
 25,
 4,
 15,
 19,
 1,
 23,
 4,
 9,
 1,
 1,
 23,
 24,
 10,
 15,
 19,
 15,
 11,
 27,
 33,
 26,
 1,
 4,
 1,
 19,
 34,
 1,
 31,
 4,
 21,
 19,
 32,
 14,
 1,
 1,
 22,
 16,
 1,
 1,
 18,
 15,
 1,
 4,
 4,
 19,
 20,
 15,
 15,
 1,
 13,
 4,
 18,
 4,
 2,
 1,
 35,
 33,
 35,
 1,
 36,
 15,
 15,
 1,
 14,
 4,
 17,
 4,
 36,
 18,
 2,
 1,
 4,
 1,
 1,
 4,
 1,
 1,
 2,
 34,
 15,
 1,
 1,
 33,
 18,
 18,
 1,
 23,
 4,
 6,
 30]

In [29]:
for comm in cluster:
    comm_name = [ g.vs[index]['name'] for index in comm]
    print(" ".join(comm_name))
    print("==================")

火腿青蔬洋芋沙拉 千島醬
紫米紅豆湯圓 （新）銷售用購物袋１８號袋 蔥爆牛肉燴飯 金牌台啤罐裝（６入） 純喫茶紅茶 七星１０毫克硬盒香煙 日式蒜香燒豚飯 冷泡茶冰釀烏龍 台鹽海洋鹼性離子水 金蘋果調味乳蘋果風味 麥香紅茶ＴＰ３００ 可口可樂ＰＥＴ 泡沫綠茶ＴＰ３００ 統一大布丁（雞蛋口味） 麥香錫蘭奶茶 金牌台灣啤酒５００ＭＬ 飲冰室茶集紅奶茶 伯朗咖啡 飲冰室茶集綠奶茶 天然水 奶油雞排歐姆蛋燴飯 冷泡茶冷萃綠茶無糖 純喫茶綠茶６５０ｍｌ 衛生冰 （新）４５號銷售用購物袋 麥香阿薩姆奶茶 全家克林姆麵包 金牌台灣啤酒 泡沫紅茶ＴＰ３００ 麥香綠茶ＴＰ３００ 雲絲頓紅１０毫克香煙 茶裏王日式綠茶 純喫茶無糖綠茶６５０ｍｌ 咖啡廣場奶香特調咖啡 麥香奶茶ＴＰ３００ 藍山咖啡 天然水２．２Ｌ 原萃日式綠茶
合庫代１５ 代收手續費１５ 中信外１５ 玉山淘寶款 雅虎拍賣繳費
台鐵取票 台鐵手續費
蕃薯（３０元） 義美奶茶 蕃薯（２５元） 鮮奶茶 頂級鮮奶優格－莓果穀物脆片 商店街取件 特濃黑可可 特濃咖啡拿鐵 統一陽光無糖高纖豆漿 大口奶油蕈菇起司雞排飯糰 熱美式中杯 熱拿鐵小杯 蕃薯（１５元） 香蕉單入 鹼性離子水 茶葉蛋（銷售用） 養樂多 簡單點無加糖優酪乳 肉鬆起酥麵包 蕃薯（２０元） 蝦皮取件Ｃ 簡單點原味優酪乳 大口法香烤雞飯糰 熱拿鐵中杯 綠豆沙牛乳 義美古早傳統無糖豆奶 特濃抹茶拿鐵
台中裁即銷罰 交通罰鍰代收即銷手續費
光泉米漿 光泉無加糖鮮豆漿 ＬＣＡ活菌原味發酵乳 肉鬆飯糰 林鳳營全脂鮮乳 鮪魚飯糰 雪花蛋糕
國光購票 國光客運手續費
黑胡椒熱狗 全家熱狗麵包 經典原味熱狗 爆濃起司熱狗
統聯客運手續費 統聯客運購票
芒果冰茶 葡萄冰茶 蘋果冰茶
促銷券０６ 頑皮滷蛋－原味 Ｃｒｅａｍ－Ｏ黑巧克力三明治餅 伊藤園蘋果紅茶
ＦＰ店到店 店到店ＦＰ手續費
代收手續費１０ 遠銀一般代Ｚ
ｅＴａｇ繳費 通行繳費 代收手續費５
中信有線代 代收手續費４ 中華電信 國民年金代 台灣自來水 台新信用卡 勞保費代收 地方查核稅款 欣林瓦斯費 健保費代收 台灣大哥大 玉山信用卡 花旗信用卡 國泰世華卡 勞工退休金 欣中天然氣 遠傳電信 台灣電力 台中二段停 聯邦信用卡
台中裁罰單 台銀學雜費 代收手續費６
高鐵手續費 高鐵取票
海鮮魚卵棒 蟹肉糰子 筊白筍 關東煮本舖拉麵

In [30]:
cluster.modularity

0.8481971436795138

In [31]:
items = []
for index, value in enumerate(g.betweenness(weights='weight')):
    if value > 0:
        items.append({ 'name': g.vs[index]['name'], 'betweeness': value })
items.sort(key=lambda x: x['betweeness'], reverse=True)

In [32]:
items[0:10]

[{'name': '（新）銷售用購物袋１８號袋', 'betweeness': 2876.0},
 {'name': '茶葉蛋（銷售用）', 'betweeness': 2741.0},
 {'name': '促銷券０６', 'betweeness': 726.0},
 {'name': '代收折價卷', 'betweeness': 506.0},
 {'name': '熱拿鐵中杯', 'betweeness': 419.0},
 {'name': '麥香奶茶ＴＰ３００', 'betweeness': 340.0},
 {'name': '大口奶油蕈菇起司雞排飯糰', 'betweeness': 255.0},
 {'name': '經典原味熱狗', 'betweeness': 255.0},
 {'name': '肉鬆飯糰', 'betweeness': 248.0},
 {'name': '鮪魚飯糰', 'betweeness': 173.0}]

In [33]:
for index, vertex in enumerate(g.vs):
    vertex.update_attributes({ 'community': cluster.membership[index], 'id': index })

In [34]:
def normalizer(max_degree):
    max_value = max_degree
    min_value = 1
    def normalize(value):
        return (value - min_value) / max_value + 1
    return normalize

In [35]:
import json
def to_json(graph, cluster):
    for index, vertex in enumerate(graph.vs):
        vertex.update_attributes({ 'community': cluster.membership[index], 'id': index })
    norm= normalizer(graph.maxdegree())
    nodes = []
    edges = []
    for edge in graph.es:
        edge_attr = {}
        edge_attr['from'], edge_attr['to'] = edge.tuple
        edge_attr['weight'] = edge['weight']
        edges.append(edge_attr)
    for node in graph.vs:
        node_attr = {}
        node_attr = { key: node[key] for key in node.attributes()}
        node_attr['degree'] = node.degree()
        nodes.append(node_attr)
    return json.dumps({
        'nodes': nodes,
        'edges': edges,
    }, indent=4)

In [36]:
data = to_json(g, cluster)

In [37]:
with open('data.json', 'w', encoding='utf-8') as file:
    file.write(data)